

# Loading and Processing Alcohol Consumption Data in Google Colab

This code mounts Google Drive in Colab, loads an alcohol consumption dataset, processes it to aggregate data for specific beverages, and filters for certain years and beverage types.




In [1]:
import os
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

folder_path = '/content/drive/MyDrive/EDA_project_phase2'

alcoholCons_path = os.path.join(folder_path, 'Per Capita US Alcohol Consumption export 2024-11-11 23-12-25.csv')

alcoholCons = pd.read_csv(alcoholCons_path, encoding='ISO-8859-1')

sum_beverages = alcoholCons[alcoholCons['Beverage_Type'].isin(['Spirits', 'Wine', 'Beer'])].groupby(['State', 'Year'])['Gallons_of_Beverage'].sum()

alcoholCons.loc[alcoholCons['Beverage_Type'] == 'All Beverages', 'Gallons_of_Beverage'] = alcoholCons[alcoholCons['Beverage_Type'] == 'All Beverages'].set_index(['State', 'Year']).index.map(sum_beverages)

alcoholCons = alcoholCons[
    (alcoholCons['Year'] >= 2016) &
    (alcoholCons['Year'] <= 2021) &
    (alcoholCons['Beverage_Type'] == "All Beverages")
]

Mounted at /content/drive


# Loading the U.S. Accidents Dataset

This code snippet demonstrates how to load the U.S. Accidents dataset from the Hugging Face `datasets` library, convert it into a Pandas DataFrame, and install any required packages.

In [2]:
!pip install datasets

from datasets import load_dataset

dataset = load_dataset("nateraw/us-accidents", split="train")

df = dataset.to_pandas()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.10k [00:00<?, ?B/s]

US_Accidents_Dec21_updated.csv:   0%|          | 0.00/1.15G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2845342 [00:00<?, ? examples/s]

# Processing Accident Data by State and Year

This code snippet demonstrates how to process the accident data by converting date information, handling missing values, extracting year, and aggregating accident counts by state and year.

In [3]:
import pandas as pd
df["Start_Time"] = pd.to_datetime(df["Start_Time"], errors='coerce')

df = df.dropna(subset=["Start_Time"])

df["Year"] = df["Start_Time"].dt.year

accidents_by_state_and_year = df.groupby(["State", "Year"]).size().reset_index(name="Accident_Count")

<ipython-input-3-1bbbf44b9749>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Year"] = df["Start_Time"].dt.year


# Mapping State Abbreviations to Full Names in Accident Data

This code snippet replaces U.S. state abbreviations with their full names in the accident dataset, making the data easier to interpret.


In [4]:
state_abbreviation_to_full_name = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DC': 'District of Columbia', 'DE': 'Delaware',
    'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois',
    'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana',
    'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota',
    'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada',
    'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York',
    'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon',
    'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota',
    'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia',
    'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}

accidents_by_state_and_year['State'] = accidents_by_state_and_year['State'].replace(state_abbreviation_to_full_name)

# Filtering Accident Data for Specific States

This list, `states_to_keep`, defines the U.S. states to include in the analysis, providing flexibility to focus on a subset of states if needed.

In [5]:
states_to_keep=['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'
 'Connecticut' 'Delaware' 'District of Columbia' 'Florida' 'Georgia'
 'Hawaii' 'Idaho' 'Illinois' 'Indiana' 'Iowa' 'Kansas' 'Kentucky'
 'Louisiana' 'Maine' 'Maryland' 'Massachusetts' 'Michigan' 'Minnesota'
 'Mississippi' 'Missouri' 'Montana' 'Nebraska' 'Nevada' 'New Hampshire'
 'New Jersey' 'New Mexico' 'New York' 'North Carolina' 'North Dakota'
 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania' 'Rhode Island' 'South Carolina'
 'South Dakota' 'Tennessee' 'Texas' 'Utah' 'Vermont' 'Virginia'
 'Washington' 'West Virginia' 'Wisconsin' 'Wyoming']

# Merging Alcohol Consumption and Accident Data by State and Year

This code snippet merges two datasets, one on alcohol consumption and the other on accident counts, by state and year. It then filters out regional aggregate rows, focusing on specific states.

In [6]:
alcoholCons_selected = alcoholCons[['State', 'Year', 'Beverage_Type','Gallons_of_Beverage', 'Gallons_of_Ethanol']]

accidents_by_state_and_year_selected = accidents_by_state_and_year[['State', 'Year', 'Accident_Count']]

merged_df = pd.merge(alcoholCons_selected, accidents_by_state_and_year_selected, on=['State', 'Year'], how='left')

states_to_drop = ['Northeast Region', 'Midwest Region', 'South Region', 'West Region', 'United States']

merged_df_filtered = merged_df[~merged_df['State'].isin(states_to_drop)]

print(merged_df_filtered)

             State  Year  Beverage_Type  Gallons_of_Beverage  \
0          Alabama  2021  All Beverages         1.107210e+08   
1           Alaska  2021  All Beverages         1.814905e+07   
2          Arizona  2021  All Beverages         1.778669e+08   
3         Arkansas  2021  All Beverages         6.182710e+07   
4       California  2021  All Beverages         1.048839e+09   
..             ...   ...            ...                  ...   
326       Virginia  2016  All Beverages         1.916542e+08   
327     Washington  2016  All Beverages         1.638047e+08   
328  West Virginia  2016  All Beverages         4.397014e+07   
329      Wisconsin  2016  All Beverages         1.741276e+08   
330        Wyoming  2016  All Beverages         1.502594e+07   

     Gallons_of_Ethanol  Accident_Count  
0               8689449          9664.0  
1               1727627             NaN  
2              14485013         24876.0  
3               4888239          6023.0  
4              922489